In [1]:
import cv2
import carla
import carla_gym
import torch
import numpy as np
import subprocess
import os
import time
import PIL
import PIL.Image
from utils import display_utils
from carla_gym.carla_multi_agent_env import CarlaMultiAgentEnv



from carla_gym import CARLA_GYM_ROOT_DIR
from carla_gym.carla_multi_agent_env import CarlaMultiAgentEnv
from carla_gym.utils import config_utils
import json


def build_all_tasks(carla_map):
    assert carla_map in ['Town01', 'Town02', 'Town03', 'Town04', 'Town05', 'Town06']
    num_zombie_vehicles = {
        'Town01': 120,
        'Town02': 70,
        'Town03': 70,
        'Town04': 150,
        'Town05': 120,
        'Town06': 120
    }[carla_map]
    num_zombie_walkers = {
        'Town01': 120,
        'Town02': 70,
        'Town03': 70,
        'Town04': 80,
        'Town05': 120,
        'Town06': 80
    }[carla_map]
    
    num_zombie_vehicles = 0
    num_zombie_walkers = 0

    weather = 'ClearNoon'
    description_folder = CARLA_GYM_ROOT_DIR / 'envs/scenario_descriptions/LeaderBoard' / carla_map

    actor_configs_dict = json.load(open(description_folder / 'actors.json'))
    route_descriptions_dict = config_utils.parse_routes_file(description_folder / 'routes.xml')

    all_tasks = []
    for route_id, route_description in route_descriptions_dict.items():
        task = {
            'weather': weather,
            'description_folder': description_folder,
            'route_id': route_id,
            'num_zombie_vehicles': num_zombie_vehicles,
            'num_zombie_walkers': num_zombie_walkers,
            'ego_vehicles': {
                'routes': route_description['ego_vehicles'],
                'actors': actor_configs_dict['ego_vehicles'],
            },
            'scenario_actors': {
                'routes': route_description['scenario_actors'],
                'actors': actor_configs_dict['scenario_actors']
            } if 'scenario_actors' in actor_configs_dict else {}
        }
        all_tasks.append(task)

    return all_tasks
    


def kill_carla(port=2005):
    # This one only kills processes linked to a certain port
    kill_process = subprocess.Popen(f'fuser -k {port}/tcp', shell=True)
    kill_process.wait()
    print(f"Killed Carla Servers on port {port}!")
    time.sleep(1)

class CarlaServerManager:
    def __init__(self, carla_sh_str, port=2000, fps=25, display=False, t_sleep=5):
        self._carla_sh_str = carla_sh_str
        self._port = port
        self._fps = fps
        self._t_sleep = t_sleep
        self._server_process = None
        self._display = display

    def start(self):
        self.stop()
#         cmd = f'bash {self._carla_sh_str} ' \
#             f'-fps={self._fps} -nosound -quality-level=Epic -carla-rpc-port={self._port}'
        cmd = f'bash {self._carla_sh_str} ' \
            f'-fps={self._fps} -nosound -quality-level=Low -carla-rpc-port={self._port}'
        if not self._display:
            cmd += ' -RenderOffScreen'

        self._server_process = subprocess.Popen(cmd, shell=True, preexec_fn=os.setsid)
        time.sleep(self._t_sleep)

    def stop(self):
        if self._server_process is not None:
            self._server_process.kill()
        kill_carla(self._port)
        time.sleep(self._t_sleep)
        

obs_configs = {
    'hero': {
#         'speed': {'module': 'actor_state.speed'}, 'gnss': {'module': 'navigation.gnss'},
#                      'location': [-1.5, 0.0, 2.0], 'rotation': [0.0, 0.0, 0.0]},
#         'route_plan': {'module': 'navigation.waypoint_plan', 'steps': 20},
        'birdview': {
            'module': 'birdview.chauffeurnet', 'width_in_pixels': 192,
            'pixels_ev_to_bottom': 32, 'pixels_per_meter': 5.0,
            'history_idx': [-16, -11, -6, -1], 'scale_bbox': True, 'scale_mask_col': 1.0}},
#     'hero': {}
}

reward_configs = {'hero': {'entry_point': 'reward.valeo_action:ValeoAction'}}
terminal_configs = {'hero': {'entry_point': 'terminal.leaderboard:Leaderboard'}}
env_configs = {'carla_map': 'Town01', 'routes_group': None, 'weather_group': 'new'}

    
def main():
    tasks = build_all_tasks(env_configs['carla_map'])
    
    server_manager = CarlaServerManager(
        '/home/carla/CarlaUE4.sh', port=2000, fps=25, display=False, t_sleep=10.)
    server_manager.start()

    env = CarlaMultiAgentEnv(
        carla_map=env_configs['carla_map'],
        host='localhost',
        port=2000,
        seed=2021,
        no_rendering=True,
        obs_configs=obs_configs,
        reward_configs=reward_configs,
        terminal_configs=terminal_configs,
        all_tasks=tasks
    )
    
    task_idx = 0
    env.set_task_idx(task_idx)
    actor_id = 'hero'

    obs = env.reset()
    debug_frames = []
    timestamps = []
    for counter in range(200):
        raw_input = obs[actor_id]
        if counter % 20 == 0: 
            print(counter)
            settings = env._world.get_settings()
            print(settings.no_rendering_mode)
            print(raw_input.keys())

#         front_rgb = raw_input['central_rgb']['data']
        bev_rgb = raw_input['birdview']['rendered']
    
#         img = overlay_images(downsample(front_rgb, 0.6), bev_rgb, (20, 20))
        if bev_rgb is not None:
            debug_frames.append(bev_rgb)

        control_dict = {
            actor_id: carla.VehicleControl(throttle=0.8, steer=0, brake=0.)
        }
        obs, reward, done, info = env.step(control_dict)
        timestamps.append(time.time())
        
    dt = np.median(np.diff(timestamps))
    print(f"dt={dt:.2f}, FPS={1./dt:.1f}")
    print(len(debug_frames))
    if len(debug_frames):
        display_utils.make_video_in_temp(debug_frames)
        

    # env.close()
    # server_manager.stop()


main()


/home/carla/.conda/envs/mile/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  5667Killed Carla Servers on port 2000!


2000/tcp:           


4.26.2-0+++UE4+Release-4.26 522 0
Disabling core dumps.


/home/carla/.conda/envs/mile/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/carla/.conda/envs/mile/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


0
True
dict_keys(['birdview'])
20
True
dict_keys(['birdview'])
40
True
dict_keys(['birdview'])
60
True
dict_keys(['birdview'])
80
True
dict_keys(['birdview'])
100
True
dict_keys(['birdview'])
120
True
dict_keys(['birdview'])
140
True
dict_keys(['birdview'])
160
True
dict_keys(['birdview'])
180
True
dict_keys(['birdview'])
dt=0.03, FPS=37.8
200
Video is saved to  /tmp/tmpirw9bnw_/temp_video.mp4


/home/carla/CarlaUE4.sh: line 5:  5800 Killed                  "$UE4_PROJECT_ROOT/CarlaUE4/Binaries/Linux/CarlaUE4-Linux-Shipping" CarlaUE4 "$@"
